Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [13]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from tensorflow import keras

In [94]:
from sklearn.metrics import classification_report

First reload the data we generated in `1_notmnist.ipynb`.

In [14]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [15]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [16]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [115]:
def classification_print(y_true, y_pred):
    if len(np.shape(y_true)) > 1:
        y_true = np.argmax(y_true, axis=1)
    if len(np.shape(y_pred)) > 1:
        y_pred = np.argmax(y_pred, axis=1)
    print(classification_report(y_true, y_pred))

In [245]:
def train_mnist(X_train=train_dataset, y_train=train_labels, 
                X_test=test_dataset, y_test=test_labels,
                learning_rate = .1,
                dropout = False,
                dropout_rate = .5,
                nn=True,
                multiple_nn = False,
                regularizer=0., 
                epochs=2, 
                batch_size=128,
                optimizer=keras.optimizers.SGD,
                callback = [],
                validation_data=(valid_dataset, valid_labels)):

    model = keras.models.Sequential()
    model.add(keras.layers.Flatten(input_shape=(784,)))
    if nn:
        if multiple_nn:
            model.add(keras.layers.Dense(100, activation=tf.nn.relu,
                                        kernel_regularizer=keras.regularizers.l2(regularizer)))
            model.add(keras.layers.Dense(100, activation=tf.nn.relu,
                                        kernel_regularizer=keras.regularizers.l2(regularizer)))
        model.add(keras.layers.Dense(1024, activation=tf.nn.relu, 
                                     kernel_regularizer=keras.regularizers.l2(regularizer)))
        if dropout:
            model.add(keras.layers.Dropout(dropout_rate))
    model.add(keras.layers.Dense(10, activation=tf.nn.softmax, 
                                 kernel_regularizer=keras.regularizers.l2(regularizer)))
    
    model.compile(optimizer=optimizer(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    history = model.fit(X_train, y_train, 
                        epochs=epochs, 
                        batch_size=batch_size,
                        validation_data=validation_data,
                        callbacks=callback)
    print(f'Test accuracy: {model.evaluate(X_test, y_test, verbose=0)[1]*100:.2f}% with regularizer: {regularizer}')
    classification_print(y_test, model.predict(X_test))
    return model, history

In [246]:
logistic_model, logisitc_history = train_mnist(nn=False)

Train on 200000 samples, validate on 10000 samples
Epoch 1/2
200000/200000 [==============================] - 5s 26us/sample - loss: 0.6886 - acc: 0.8182 - val_loss: 0.6576 - val_acc: 0.8272
Epoch 2/2
200000/200000 [==============================] - 4s 18us/sample - loss: 0.6408 - acc: 0.8300 - val_loss: 0.6502 - val_acc: 0.8292
Test accuracy: 89.61% with regularizer: 0.0
              precision    recall  f1-score   support

           0       0.93      0.91      0.92      1000
           1       0.88      0.90      0.89      1000
           2       0.90      0.93      0.92      1000
           3       0.93      0.91      0.92      1000
           4       0.90      0.86      0.88      1000
           5       0.87      0.93      0.90      1000
           6       0.90      0.89      0.89      1000
           7       0.92      0.89      0.90      1000
           8       0.87      0.84      0.86      1000
           9       0.87      0.90      0.88      1000

    accuracy                 

In [214]:
logistic_model.summary()

Model: "sequential_92"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_91 (Flatten)         (None, 784)               0         
_________________________________________________________________
dense_136 (Dense)            (None, 10)                7850      
Total params: 7,850
Trainable params: 7,850
Non-trainable params: 0
_________________________________________________________________


In [163]:
logistic_model, logisitc_history = train_mnist(nn=False, regularizer=.01)

Train on 200000 samples, validate on 10000 samples
Epoch 1/2
200000/200000 [==============================] - 4s 21us/sample - loss: 0.8282 - acc: 0.8146 - val_loss: 0.7857 - val_acc: 0.8202
Epoch 2/2
200000/200000 [==============================] - 3s 17us/sample - loss: 0.7795 - acc: 0.8214 - val_loss: 0.7874 - val_acc: 0.8186
Test accuracy: 88.85% with regularizer:0.01
              precision    recall  f1-score   support

           0       0.92      0.89      0.91      1000
           1       0.93      0.87      0.90      1000
           2       0.91      0.93      0.92      1000
           3       0.91      0.91      0.91      1000
           4       0.91      0.86      0.89      1000
           5       0.91      0.91      0.91      1000
           6       0.90      0.89      0.90      1000
           7       0.92      0.87      0.89      1000
           8       0.80      0.86      0.83      1000
           9       0.80      0.89      0.85      1000

    accuracy                 

In [162]:
logistic_model, logisitc_history = train_mnist(nn=False, regularizer=.001)

Train on 200000 samples, validate on 10000 samples
Epoch 1/2
200000/200000 [==============================] - 4s 18us/sample - loss: 0.7152 - acc: 0.8181 - val_loss: 0.6869 - val_acc: 0.8262
Epoch 2/2
200000/200000 [==============================] - 3s 15us/sample - loss: 0.6696 - acc: 0.8290 - val_loss: 0.6739 - val_acc: 0.8304
Test accuracy: 89.71% with regularizer:0.001
              precision    recall  f1-score   support

           0       0.92      0.91      0.91      1000
           1       0.92      0.89      0.90      1000
           2       0.89      0.93      0.91      1000
           3       0.91      0.93      0.92      1000
           4       0.90      0.87      0.88      1000
           5       0.92      0.92      0.92      1000
           6       0.91      0.89      0.90      1000
           7       0.90      0.89      0.90      1000
           8       0.84      0.85      0.85      1000
           9       0.86      0.90      0.88      1000

    accuracy                

In [164]:
logistic_model, logisitc_history = train_mnist(nn=False, regularizer=1)

Train on 200000 samples, validate on 10000 samples
Epoch 1/2
200000/200000 [==============================] - 4s 19us/sample - loss: 1.9211 - acc: 0.7070 - val_loss: 1.9044 - val_acc: 0.6929
Epoch 2/2
200000/200000 [==============================] - 3s 15us/sample - loss: 1.8839 - acc: 0.7074 - val_loss: 1.8995 - val_acc: 0.6621
Test accuracy: 72.71% with regularizer:1
              precision    recall  f1-score   support

           0       0.94      0.68      0.79      1000
           1       0.79      0.74      0.76      1000
           2       0.55      0.89      0.68      1000
           3       0.89      0.80      0.84      1000
           4       0.87      0.66      0.75      1000
           5       0.93      0.76      0.84      1000
           6       0.96      0.36      0.52      1000
           7       0.92      0.66      0.77      1000
           8       0.69      0.83      0.75      1000
           9       0.45      0.89      0.60      1000

    accuracy                    

In [146]:
nn_model, nn_history = train_mnist()

Train on 200000 samples, validate on 10000 samples
Epoch 1/2
200000/200000 [==============================] - 17s 83us/sample - loss: 0.7475 - acc: 0.7993 - val_loss: 0.6105 - val_acc: 0.8295
Epoch 2/2
200000/200000 [==============================] - 16s 78us/sample - loss: 0.5760 - acc: 0.8369 - val_loss: 0.5637 - val_acc: 0.8406
Test accuracy: 90.85% with regularizer:0.0
              precision    recall  f1-score   support

           0       0.93      0.92      0.92      1000
           1       0.91      0.89      0.90      1000
           2       0.91      0.94      0.93      1000
           3       0.93      0.92      0.92      1000
           4       0.93      0.88      0.90      1000
           5       0.92      0.93      0.92      1000
           6       0.91      0.91      0.91      1000
           7       0.93      0.90      0.91      1000
           8       0.87      0.88      0.87      1000
           9       0.86      0.93      0.89      1000

    accuracy                

In [147]:
nn_model, nn_history = train_mnist(regularizer=.01)

Train on 200000 samples, validate on 10000 samples
Epoch 1/2
200000/200000 [==============================] - 19s 96us/sample - loss: 7.6053 - acc: 0.7969 - val_loss: 5.6360 - val_acc: 0.8211
Epoch 2/2
200000/200000 [==============================] - 20s 98us/sample - loss: 4.3910 - acc: 0.8280 - val_loss: 3.3957 - val_acc: 0.8275
Test accuracy: 89.37% with regularizer:0.01
              precision    recall  f1-score   support

           0       0.94      0.90      0.91      1000
           1       0.92      0.87      0.89      1000
           2       0.90      0.92      0.91      1000
           3       0.92      0.91      0.91      1000
           4       0.91      0.85      0.88      1000
           5       0.87      0.94      0.90      1000
           6       0.92      0.89      0.90      1000
           7       0.91      0.88      0.90      1000
           8       0.85      0.86      0.85      1000
           9       0.83      0.92      0.87      1000

    accuracy               

In [148]:
nn_model, nn_history = train_mnist(regularizer=.001)

Train on 200000 samples, validate on 10000 samples
Epoch 1/2
200000/200000 [==============================] - 20s 102us/sample - loss: 1.6407 - acc: 0.7981 - val_loss: 1.4793 - val_acc: 0.8293
Epoch 2/2
200000/200000 [==============================] - 20s 98us/sample - loss: 1.4250 - acc: 0.8357 - val_loss: 1.3890 - val_acc: 0.8392
Test accuracy: 90.80% with regularizer:0.001
              precision    recall  f1-score   support

           0       0.94      0.91      0.92      1000
           1       0.92      0.89      0.90      1000
           2       0.92      0.93      0.92      1000
           3       0.92      0.93      0.92      1000
           4       0.92      0.88      0.90      1000
           5       0.90      0.94      0.92      1000
           6       0.91      0.91      0.91      1000
           7       0.92      0.89      0.91      1000
           8       0.87      0.86      0.87      1000
           9       0.87      0.94      0.90      1000

    accuracy             

In [150]:
nn_model, nn_history = train_mnist(regularizer=.0001)

Train on 200000 samples, validate on 10000 samples
Epoch 1/2
200000/200000 [==============================] - 21s 107us/sample - loss: 0.8338 - acc: 0.8003 - val_loss: 0.7018 - val_acc: 0.8293
Epoch 2/2
200000/200000 [==============================] - 20s 99us/sample - loss: 0.6684 - acc: 0.8368 - val_loss: 0.6560 - val_acc: 0.8411
Test accuracy: 90.64% with regularizer:0.0001
              precision    recall  f1-score   support

           0       0.94      0.91      0.92      1000
           1       0.91      0.88      0.90      1000
           2       0.93      0.93      0.93      1000
           3       0.90      0.92      0.91      1000
           4       0.92      0.89      0.90      1000
           5       0.89      0.94      0.91      1000
           6       0.91      0.91      0.91      1000
           7       0.91      0.90      0.91      1000
           8       0.87      0.87      0.87      1000
           9       0.88      0.92      0.90      1000

    accuracy            

In [149]:
nn_model, nn_history = train_mnist(regularizer=1)

Train on 200000 samples, validate on 10000 samples
Epoch 1/2
200000/200000 [==============================] - 18s 92us/sample - loss: 17.0011 - acc: 0.1571 - val_loss: 2.3026 - val_acc: 0.1000
Epoch 2/2
200000/200000 [==============================] - 18s 89us/sample - loss: 2.3026 - acc: 0.0993 - val_loss: 2.3026 - val_acc: 0.1000
Test accuracy: 10.00% with regularizer:1
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1000
           1       0.00      0.00      0.00      1000
           2       0.00      0.00      0.00      1000
           3       0.00      0.00      0.00      1000
           4       0.10      1.00      0.18      1000
           5       0.00      0.00      0.00      1000
           6       0.00      0.00      0.00      1000
           7       0.00      0.00      0.00      1000
           8       0.00      0.00      0.00      1000
           9       0.00      0.00      0.00      1000

    accuracy                 

C:\Users\Administrator\Anaconda3\envs\nlp\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [167]:
log_model, _ = train_mnist(X_train=train_dataset[:1000,:],y_train = train_labels[:1000,:],nn=False, regularizer=.001)

Train on 1000 samples, validate on 10000 samples
Epoch 1/2
1000/1000 [==============================] - 1s 934us/sample - loss: 1.9120 - acc: 0.3740 - val_loss: 1.3868 - val_acc: 0.6379
Epoch 2/2
1000/1000 [==============================] - 0s 133us/sample - loss: 1.2468 - acc: 0.6930 - val_loss: 1.1163 - val_acc: 0.7276
Test accuracy: 79.80% with regularizer:0.001
              precision    recall  f1-score   support

           0       0.91      0.78      0.84      1000
           1       0.88      0.75      0.81      1000
           2       0.82      0.80      0.81      1000
           3       0.83      0.83      0.83      1000
           4       0.77      0.71      0.74      1000
           5       0.68      0.91      0.78      1000
           6       0.81      0.82      0.81      1000
           7       0.90      0.75      0.82      1000
           8       0.80      0.79      0.80      1000
           9       0.68      0.83      0.75      1000

    accuracy                        

In [168]:
nn_model, _ = train_mnist(X_train=train_dataset[:1000,:], y_train = train_labels[:1000,:],regularizer=.001)

Train on 1000 samples, validate on 10000 samples
Epoch 1/2
1000/1000 [==============================] - 1s 1ms/sample - loss: 2.6493 - acc: 0.4890 - val_loss: 2.1322 - val_acc: 0.7096
Epoch 2/2
1000/1000 [==============================] - 1s 537us/sample - loss: 1.9840 - acc: 0.7360 - val_loss: 1.8774 - val_acc: 0.7553
Test accuracy: 82.98% with regularizer:0.001
              precision    recall  f1-score   support

           0       0.77      0.84      0.80      1000
           1       0.90      0.81      0.85      1000
           2       0.79      0.91      0.84      1000
           3       0.81      0.88      0.84      1000
           4       0.89      0.76      0.82      1000
           5       0.76      0.92      0.83      1000
           6       0.91      0.78      0.84      1000
           7       0.91      0.80      0.85      1000
           8       0.78      0.84      0.81      1000
           9       0.85      0.77      0.80      1000

    accuracy                          

Test accuracy is lower due to overfitting.

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [146]:
nn_model, nn_history = train_mnist()

Train on 200000 samples, validate on 10000 samples
Epoch 1/2
200000/200000 [==============================] - 17s 83us/sample - loss: 0.7475 - acc: 0.7993 - val_loss: 0.6105 - val_acc: 0.8295
Epoch 2/2
200000/200000 [==============================] - 16s 78us/sample - loss: 0.5760 - acc: 0.8369 - val_loss: 0.5637 - val_acc: 0.8406
Test accuracy: 90.85% with regularizer:0.0
              precision    recall  f1-score   support

           0       0.93      0.92      0.92      1000
           1       0.91      0.89      0.90      1000
           2       0.91      0.94      0.93      1000
           3       0.93      0.92      0.92      1000
           4       0.93      0.88      0.90      1000
           5       0.92      0.93      0.92      1000
           6       0.91      0.91      0.91      1000
           7       0.93      0.90      0.91      1000
           8       0.87      0.88      0.87      1000
           9       0.86      0.93      0.89      1000

    accuracy                

In [170]:
nn_model, _ = train_mnist(dropout=True)

Train on 200000 samples, validate on 10000 samples
Epoch 1/2
200000/200000 [==============================] - 18s 89us/sample - loss: 0.5876 - acc: 0.8310 - val_loss: 0.4786 - val_acc: 0.8631
Epoch 2/2
200000/200000 [==============================] - 17s 85us/sample - loss: 0.4820 - acc: 0.8598 - val_loss: 0.4295 - val_acc: 0.8802
Test accuracy: 93.73% with regularizer:0.0
              precision    recall  f1-score   support

           0       0.94      0.95      0.95      1000
           1       0.94      0.93      0.93      1000
           2       0.94      0.96      0.95      1000
           3       0.93      0.95      0.94      1000
           4       0.96      0.92      0.94      1000
           5       0.96      0.95      0.95      1000
           6       0.92      0.94      0.93      1000
           7       0.95      0.94      0.94      1000
           8       0.93      0.90      0.91      1000
           9       0.91      0.95      0.93      1000

    accuracy                

In [171]:
nn_model, _ = train_mnist(dropout=True, regularizer=.001)

Train on 200000 samples, validate on 10000 samples
Epoch 1/2
200000/200000 [==============================] - 22s 109us/sample - loss: 1.2888 - acc: 0.8313 - val_loss: 1.0201 - val_acc: 0.8598
Epoch 2/2
200000/200000 [==============================] - 21s 104us/sample - loss: 0.9063 - acc: 0.8566 - val_loss: 0.7808 - val_acc: 0.8690
Test accuracy: 92.87% with regularizer:0.001
              precision    recall  f1-score   support

           0       0.96      0.94      0.95      1000
           1       0.92      0.93      0.92      1000
           2       0.93      0.95      0.94      1000
           3       0.93      0.93      0.93      1000
           4       0.95      0.92      0.93      1000
           5       0.93      0.95      0.94      1000
           6       0.92      0.93      0.92      1000
           7       0.97      0.91      0.94      1000
           8       0.92      0.89      0.90      1000
           9       0.87      0.95      0.91      1000

    accuracy            

Dropout did a great job on avoiding overfitting.

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [174]:
nn_model, _ = train_mnist(multiple_nn=True, dropout=True)

Train on 200000 samples, validate on 10000 samples
Epoch 1/2
200000/200000 [==============================] - 11s 55us/sample - loss: 0.5768 - acc: 0.8299 - val_loss: 0.4697 - val_acc: 0.8595
Epoch 2/2
200000/200000 [==============================] - 10s 50us/sample - loss: 0.4535 - acc: 0.8633 - val_loss: 0.4167 - val_acc: 0.8774
Test accuracy: 93.33% with regularizer: 0.0
              precision    recall  f1-score   support

           0       0.96      0.94      0.95      1000
           1       0.94      0.92      0.93      1000
           2       0.94      0.94      0.94      1000
           3       0.93      0.95      0.94      1000
           4       0.93      0.93      0.93      1000
           5       0.93      0.95      0.94      1000
           6       0.91      0.94      0.92      1000
           7       0.95      0.93      0.94      1000
           8       0.92      0.91      0.91      1000
           9       0.94      0.93      0.93      1000

    accuracy               

let's try more epochs.

In [179]:
nn_model, _ = train_mnist(multiple_nn=True, dropout=True, epochs=10)

Train on 200000 samples, validate on 10000 samples
Epoch 1/10
200000/200000 [==============================] - 12s 61us/sample - loss: 0.5645 - acc: 0.8308 - val_loss: 0.4580 - val_acc: 0.8623
Epoch 2/10
200000/200000 [==============================] - 11s 56us/sample - loss: 0.4304 - acc: 0.8681 - val_loss: 0.4001 - val_acc: 0.8765
Epoch 3/10
200000/200000 [==============================] - 11s 56us/sample - loss: 0.3883 - acc: 0.8813 - val_loss: 0.3762 - val_acc: 0.8860
Epoch 4/10
200000/200000 [==============================] - 11s 56us/sample - loss: 0.3602 - acc: 0.8896 - val_loss: 0.3704 - val_acc: 0.8841
Epoch 5/10
200000/200000 [==============================] - 11s 56us/sample - loss: 0.3406 - acc: 0.8947 - val_loss: 0.3525 - val_acc: 0.8917
Epoch 6/10
200000/200000 [==============================] - 11s 56us/sample - loss: 0.3246 - acc: 0.8995 - val_loss: 0.3448 - val_acc: 0.8948
Epoch 7/10
200000/200000 [==============================] - 11s 56us/sample - loss: 0.3113 - acc:

good job. we have time.

now try decay learning rate.

In [192]:
initial_learning_rate = .5
lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate,
                                                        decay_steps = 10000,
                                                        decay_rate = .96,
                                                        staircase = True)

In [193]:
nn_model, _ = train_mnist(multiple_nn=True, dropout=True, epochs=10, learning_rate=lr_schedule)

Train on 200000 samples, validate on 10000 samples
Epoch 1/10
200000/200000 [==============================] - 13s 65us/sample - loss: 0.5070 - acc: 0.8439 - val_loss: 0.4151 - val_acc: 0.8729
Epoch 2/10
200000/200000 [==============================] - 12s 58us/sample - loss: 0.3918 - acc: 0.8790 - val_loss: 0.3961 - val_acc: 0.8812
Epoch 3/10
200000/200000 [==============================] - 12s 58us/sample - loss: 0.3559 - acc: 0.8899 - val_loss: 0.3627 - val_acc: 0.8873
Epoch 4/10
200000/200000 [==============================] - 12s 58us/sample - loss: 0.3333 - acc: 0.8958 - val_loss: 0.3438 - val_acc: 0.8949
Epoch 5/10
200000/200000 [==============================] - 11s 57us/sample - loss: 0.3168 - acc: 0.9010 - val_loss: 0.3456 - val_acc: 0.8939
Epoch 6/10
200000/200000 [==============================] - 11s 57us/sample - loss: 0.3048 - acc: 0.9042 - val_loss: 0.3461 - val_acc: 0.8954
Epoch 7/10
200000/200000 [==============================] - 11s 57us/sample - loss: 0.2925 - acc:

didn't change much.

now try more epochs with callbacks.

In [230]:
# EarlyStopping's monitor should use validation_data's params, like val_loss or val_acc.
callback = keras.callbacks.EarlyStopping(monitor='val_acc', mode='max', patience=2)
checkpoint = keras.callbacks.ModelCheckpoint('checkpoint/nn_model.ckpt',
                                            monitor='val_acc',
                                            verbose=1,
                                            save_best_only=True,
                                            load_weights_on_restart=True)

In [249]:
model, _ = train_mnist(multiple_nn=True, dropout=True, epochs=100, 
                          learning_rate=lr_schedule, 
                          callback=[callback, checkpoint])

Train on 200000 samples, validate on 10000 samples
Epoch 1/100
199680/200000 [============================>.] - ETA: 0s - loss: 0.2192 - acc: 0.9306
Epoch 00001: val_acc did not improve from 0.90270
200000/200000 [==============================] - 14s 69us/sample - loss: 0.2192 - acc: 0.9306 - val_loss: 0.4390 - val_acc: 0.9010
Epoch 2/100
199296/200000 [============================>.] - ETA: 0s - loss: 0.2215 - acc: 0.9296
Epoch 00002: val_acc did not improve from 0.90270
200000/200000 [==============================] - 12s 61us/sample - loss: 0.2217 - acc: 0.9295 - val_loss: 0.4183 - val_acc: 0.9015
Epoch 3/100
199936/200000 [============================>.] - ETA: 0s - loss: 0.2172 - acc: 0.9310
Epoch 00003: val_acc did not improve from 0.90270
200000/200000 [==============================] - 12s 61us/sample - loss: 0.2172 - acc: 0.9310 - val_loss: 0.4455 - val_acc: 0.9001
Epoch 4/100
199168/200000 [============================>.] - ETA: 0s - loss: 0.2198 - acc: 0.9303
Epoch 00004: v

looks like 95% is the best?

try another optimizer.

In [252]:
model.fit(train_dataset, train_labels, validation_data=(valid_dataset, valid_labels), callbacks=[callback, checkpoint])

Train on 200000 samples, validate on 10000 samples
 13824/200000 [=>............................] - ETA: 28s - loss: nan - acc: 0.1427

KeyboardInterrupt: 

In [238]:
model.save('test.h5')
model.summary()

Model: "sequential_97"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_96 (Flatten)         (None, 784)               0         
_________________________________________________________________
dense_153 (Dense)            (None, 100)               78500     
_________________________________________________________________
dense_154 (Dense)            (None, 100)               10100     
_________________________________________________________________
dense_155 (Dense)            (None, 1024)              103424    
_________________________________________________________________
dropout_15 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_156 (Dense)            (None, 10)                10250     
Total params: 202,274
Trainable params: 202,274
Non-trainable params: 0
_______________________________________________

further training on existing model.

In [242]:
new_model = keras.models.load_model('test.h5')

In [235]:
np.testing.assert_allclose(model.predict(train_dataset),
                          new_model.predict(train_dataset))

In [243]:
new_model.summary()

Model: "sequential_97"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_96 (Flatten)         (None, 784)               0         
_________________________________________________________________
dense_153 (Dense)            (None, 100)               78500     
_________________________________________________________________
dense_154 (Dense)            (None, 100)               10100     
_________________________________________________________________
dense_155 (Dense)            (None, 1024)              103424    
_________________________________________________________________
dropout_15 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_156 (Dense)            (None, 10)                10250     
Total params: 202,274
Trainable params: 202,274
Non-trainable params: 0
_______________________________________________

In [244]:
new_model.evaluate(test_dataset, test_labels)

10000/10000 [==============================] - 1s 97us/sample - loss: 0.2258 - acc: 0.9542


[0.22576687259974715, 0.9542]